# Modelagem Preditiva - Churn Prediction

Neste notebook, vamos construir e avaliar modelos de machine learning para prever o cancelamento de clientes com base em seus dados históricos.

Como o dataset original apresenta uma proporção elevada de cancelamentos (ao contrário da realidade da maioria dos negócios), adotamos **duas abordagens paralelas**:

- **Abordagem A:** Usar o dataset original como está (com alto índice de churn).
- **Abordagem B:** Simular uma base mais realista, com churn em aproximadamente 30% dos casos.

Com isso, buscamos observar o impacto da distribuição da variável alvo no desempenho dos modelos, simulando também uma situação mais próxima do mercado real.

In [2]:
#  Importação de bibliotecas
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, accuracy_score

import warnings
warnings.filterwarnings("ignore")


In [3]:
# Carregando o dataset limpo
df = pd.read_csv('../data/raw/cancelamentos.csv')
df = df.dropna()  # Garantindo que não tem valores ausentes

In [4]:
# Simulando cenário mais realista (30% churn, 70% não churn)

# 1. Filtrar as duas classes
df_churn = df[df['cancelou'] == 1]
df_nao_churn = df[df['cancelou'] == 0]

# 2. Quantos churners precisamos para que eles representem 30% da base?
n_nao_churn = len(df_nao_churn)
n_churn_necessario = int((30 / 70) * n_nao_churn)

# 3. Sample dos churners baseado nessa quantidade
df_churn_amostrado = df_churn.sample(n=n_churn_necessario, random_state=42)

# 4. Juntar a base simulada
df_simulado = pd.concat([df_churn_amostrado, df_nao_churn]).sample(frac=1, random_state=42)


In [5]:
# Função de pré-processamento (para ambas as bases)

def preparar_dados(df):
    df_temp = df.copy()
    
    # Encoding binário para 'sexo'
    le = LabelEncoder()
    df_temp['sexo'] = le.fit_transform(df_temp['sexo'])
    
    # One-hot encoding para 'assinatura' e 'duracao_contrato'
    df_temp = pd.get_dummies(df_temp, columns=['assinatura', 'duracao_contrato'], drop_first=True)
    
    # Separar X e y
    X = df_temp.drop(['cancelou', 'CustomerID'], axis=1)
    y = df_temp['cancelou']
    
    return X, y


In [6]:
# Aplicando a função nas duas bases

# Abordagem A - Base original
X, y = preparar_dados(df)

# Abordagem B - Base simulada com 30% churn
X_sim, y_sim = preparar_dados(df_simulado)


In [7]:
# Separando treino e teste

# Base original
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

# Base simulada
X_train_sim, X_test_sim, y_train_sim, y_test_sim = train_test_split(
    X_sim, y_sim, test_size=0.25, stratify=y_sim, random_state=42
)


In [8]:
# Balanceamento com SMOTE (equilibrando mais as bases de treino)

sm = SMOTE(random_state=42)

X_train_res, y_train_res = sm.fit_resample(X_train, y_train)           # original
X_train_sim_res, y_train_sim_res = sm.fit_resample(X_train_sim, y_train_sim)  # simulado


## Comparação de Modelos

A seguir, comparamos 5 algoritmos de classificação aplicados em duas versões da base:

- Base original com alta proporção de churners.
- Base simulada com 30% de churn, mais próxima do mercado real.

As métricas utilizadas foram:
- Acurácia
- F1-score
- ROC AUC

O objetivo é entender qual modelo se comporta melhor em diferentes cenários e qual oferece maior capacidade preditiva para problemas reais.


In [9]:
# Definindo os modelos

modelos = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "SGD Classifier (SVM Linear)": SGDClassifier(loss="hinge", random_state=42, max_iter=1000, tol=1e-3)
}

In [10]:
# Treinando os modelos

def avaliar_modelos(modelos, X_train, y_train, X_test, y_test):
    resultados = []

    for nome, modelo in modelos.items():
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        try:
            y_proba = modelo.predict_proba(X_test)[:, 1]
        except AttributeError:
            # fallback para modelos que não têm predict_proba
            y_proba = modelo.decision_function(X_test)
            # se o resultado for um array 1D, normalizamos para 0 a 1
            y_proba = (y_proba - y_proba.min()) / (y_proba.max() - y_proba.min())

        f1 = f1_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_proba)

        print(f"\n📊 Modelo: {nome}")
        print(f"Acurácia: {acc:.3f}")
        print(f"F1-score: {f1:.3f}")
        print(f"ROC AUC: {auc:.3f}")
        print("Classification Report:\n", classification_report(y_test, y_pred))

        resultados.append({
            "Modelo": nome,
            "Acurácia": acc,
            "F1-score": f1,
            "ROC AUC": auc
        })

    return pd.DataFrame(resultados).sort_values(by="F1-score", ascending=False)



In [13]:
# Avaliando a base original balanceada com SMOTE

avaliacao_original = avaliar_modelos(modelos, X_train_res, y_train_res, X_test, y_test)
avaliacao_original


📊 Modelo: Logistic Regression
Acurácia: 0.894
F1-score: 0.903
ROC AUC: 0.958
Classification Report:
               precision    recall  f1-score   support

         0.0       0.85      0.92      0.88     95417
         1.0       0.94      0.87      0.90    124998

    accuracy                           0.89    220415
   macro avg       0.89      0.90      0.89    220415
weighted avg       0.90      0.89      0.89    220415


📊 Modelo: Decision Tree
Acurácia: 1.000
F1-score: 1.000
ROC AUC: 1.000
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     95417
         1.0       1.00      1.00      1.00    124998

    accuracy                           1.00    220415
   macro avg       1.00      1.00      1.00    220415
weighted avg       1.00      1.00      1.00    220415


📊 Modelo: Random Forest
Acurácia: 1.000
F1-score: 1.000
ROC AUC: 1.000
Classification Report:
               precision    recall  f1-score   suppor

,Modelo,Acurácia,F1-score,ROC AUC
1,Decision Tree,0.999955,0.999960,0.999955
2,Random Forest,0.999946,0.999952,1.000000
3,Gradient Boosting,0.996851,0.997216,0.999676
4,SGD Classifier (SVM Linear),0.893279,0.904357,0.958911
0,Logistic Regression,0.894254,0.903466,0.957938


In [14]:
# Avaliando a base simulada com 30% de churn

avaliacao_simulada = avaliar_modelos(modelos, X_train_sim_res, y_train_sim_res, X_test_sim, y_test_sim)
avaliacao_simulada



📊 Modelo: Logistic Regression
Acurácia: 0.885
F1-score: 0.811
ROC AUC: 0.935
Classification Report:
               precision    recall  f1-score   support

         0.0       0.92      0.91      0.92     95417
         1.0       0.80      0.82      0.81     40893

    accuracy                           0.88    136310
   macro avg       0.86      0.87      0.86    136310
weighted avg       0.89      0.88      0.89    136310


📊 Modelo: Decision Tree
Acurácia: 1.000
F1-score: 1.000
ROC AUC: 1.000
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     95417
         1.0       1.00      1.00      1.00     40893

    accuracy                           1.00    136310
   macro avg       1.00      1.00      1.00    136310
weighted avg       1.00      1.00      1.00    136310


📊 Modelo: Random Forest
Acurácia: 1.000
F1-score: 0.999
ROC AUC: 1.000
Classification Report:
               precision    recall  f1-score   suppor

,Modelo,Acurácia,F1-score,ROC AUC
1,Decision Tree,0.999861,0.999768,0.999852
2,Random Forest,0.999508,0.999180,1.000000
3,Gradient Boosting,0.996977,0.994937,0.999513
4,SGD Classifier (SVM Linear),0.906427,0.816245,0.954003
0,Logistic Regression,0.884814,0.811116,0.935495


## Salvando o Modelo Final

Após comparar o desempenho de diferentes algoritmos, escolhemos o **Random Forest** como modelo final para deploy, com base no seu alto desempenho em F1-score, estabilidade e interpretabilidade.

A seguir, treinamos o modelo final com a base simulada balanceada (com ~30% de churn) e salvamos o artefato em formato `.pkl`, que poderá ser usado em aplicações web, notebooks de previsão ou APIs.

O arquivo será salvo na pasta `outputs/models/`.


In [16]:
import joblib
import os

# Cria a pasta se não existir
os.makedirs("outputs/models", exist_ok=True)

# Utilizando base simulada (que é mais realista)
modelo_final = RandomForestClassifier(random_state=42)
modelo_final.fit(X_train_sim_res, y_train_sim_res)

joblib.dump(modelo_final, "outputs/models/modelo_random_forest.pkl")

['outputs/models/modelo_random_forest.pkl']